In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv("2024_tw_posts_president_scored_anon.csv")
print("Loaded dataset with shape:", df.shape)

# STEP 1: Drop unnecessary ID columns with too many nulls
df.drop(columns=["quoteId", "inReplyToId"], inplace=True)

# STEP 2: Drop rows where all message/topic-related illuminating columns are missing
illuminating_cols = [col for col in df.columns if 'illuminating' in col and col not in ['illuminating_scored_message']]
df = df.dropna(subset=illuminating_cols, how='all')

# STEP 3: Convert 'createdAt' to datetime
df['createdAt'] = pd.to_datetime(df['createdAt'], errors='coerce')

# STEP 4: Optional - Filter only English tweets (if needed)
df = df[df['lang'] == 'en']

# STEP 5: Reset index after filtering
df.reset_index(drop=True, inplace=True)

# STEP 6: Final summary
print("Cleaned dataset shape:", df.shape)
print("\nRemaining nulls:\n", df.isnull().sum()[df.isnull().sum() > 0])

# STEP 7: Save cleaned version (optional)
df.to_csv("tw_posts_cleaned.csv", index=False)


Loaded dataset with shape: (27304, 47)
Cleaned dataset shape: (27281, 45)

Remaining nulls:
 election_integrity_Truth_illuminating        1267
advocacy_msg_type_illuminating               1267
issue_msg_type_illuminating                  1267
attack_msg_type_illuminating                 1267
image_msg_type_illuminating                  1267
cta_msg_type_illuminating                    1267
engagement_cta_subtype_illuminating          1267
fundraising_cta_subtype_illuminating         1267
voting_cta_subtype_illuminating              1267
covid_topic_illuminating                     1267
economy_topic_illuminating                   1267
education_topic_illuminating                 1267
environment_topic_illuminating               1267
foreign_policy_topic_illuminating            1267
governance_topic_illuminating                1267
health_topic_illuminating                    1267
immigration_topic_illuminating               1267
lgbtq_issues_topic_illuminating              1267
militar

In [3]:
import csv
import math
from collections import defaultdict, Counter
import pandas as pd

# Load the CSV file
file_path = "tw_posts_cleaned.csv"
with open(file_path, mode='r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    data = list(reader)

# Helper functions
def mean(values):
    return sum(values) / len(values) if values else None

def std_dev(values, m):
    return math.sqrt(sum((x - m) ** 2 for x in values) / len(values)) if values else None

# Classify columns
sample_row = data[0]
numeric_columns = []
non_numeric_columns = []
for key, value in sample_row.items():
    try:
        float(value)
        numeric_columns.append(key)
    except:
        non_numeric_columns.append(key)

# Overall numeric stats
numeric_stats = {}
for col in numeric_columns:
    values = [float(row[col]) for row in data if row[col]]
    if values:
        m = mean(values)
        numeric_stats[col] = {
            "count": len(values),
            "mean": m,
            "min": min(values),
            "max": max(values),
            "std_dev": std_dev(values, m)
        }

# Non-numeric stats
non_numeric_stats = {}
for col in non_numeric_columns:
    values = [row[col] for row in data if row[col]]
    counter = Counter(values)
    most_common = counter.most_common(1)[0] if counter else ("", 0)
    non_numeric_stats[col] = {
        "count": len(values),
        "unique": len(counter),
        "most_frequent": most_common[0],
        "frequency": most_common[1]
    }

# Grouped stats by source
grouped_by_source = defaultdict(list)
for row in data:
    grouped_by_source[row["source"]].append(row)

grouped_source_stats = {}
for source, rows in grouped_by_source.items():
    stats = {}
    for col in numeric_columns:
        values = [float(row[col]) for row in rows if row[col]]
        if values:
            m = mean(values)
            stats[col] = {
                "count": len(values),
                "mean": m,
                "min": min(values),
                "max": max(values),
                "std_dev": std_dev(values, m)
            }
    grouped_source_stats[source] = stats

# Grouped stats by (source, id)
grouped_by_source_id = defaultdict(list)
for row in data:
    key = (row["source"], row["id"])
    grouped_by_source_id[key].append(row)

grouped_source_id_stats = {}
for key, rows in grouped_by_source_id.items():
    stats = {}
    for col in numeric_columns:
        values = [float(row[col]) for row in rows if row[col]]
        if values:
            m = mean(values)
            stats[col] = {
                "count": len(values),
                "mean": m,
                "min": min(values),
                "max": max(values),
                "std_dev": std_dev(values, m)
            }
    grouped_source_id_stats[key] = stats

# === Export / View in Jupyter ===
numeric_df = pd.DataFrame.from_dict(numeric_stats, orient='index')
non_numeric_df = pd.DataFrame.from_dict(non_numeric_stats, orient='index')

grouped_source_records = []
for source, stats in grouped_source_stats.items():
    flat = {"source": source}
    for col, metrics in stats.items():
        for metric, value in metrics.items():
            flat[f"{col}_{metric}"] = value
    grouped_source_records.append(flat)
grouped_source_df = pd.DataFrame(grouped_source_records)

grouped_source_id_records = []
for (source, post_id), stats in grouped_source_id_stats.items():
    flat = {"source": source, "id": post_id}
    for col, metrics in stats.items():
        for metric, value in metrics.items():
            flat[f"{col}_{metric}"] = value
    grouped_source_id_records.append(flat)
grouped_source_id_df = pd.DataFrame(grouped_source_id_records)

# Display or export
display(numeric_df)
display(non_numeric_df)
display(grouped_source_df)
display(grouped_source_id_df)



,count,mean,min,max,std_dev
retweetCount,27281,1322.428833,0.0,144615.0,3.405781e+03
replyCount,27281,1064.435431,0.0,121270.0,3.176136e+03
likeCount,27281,6913.519886,0.0,915221.0,2.159145e+04
quoteCount,27281,128.155310,0.0,123320.0,1.131985e+03
viewCount,27281,507323.401525,5.0,333502775.0,3.213430e+06
bookmarkCount,27281,136.269528,0.0,42693.0,7.128515e+02
election_integrity_Truth_illuminating,26014,0.037172,0.0,1.0,1.891838e-01
advocacy_msg_type_illuminating,26014,0.564042,0.0,1.0,4.958816e-01
issue_msg_type_illuminating,26014,0.508034,0.0,1.0,4.999354e-01
attack_msg_type_illuminating,26014,0.307834,0.0,1.0,4.615976e-01


,count,unique,most_frequent,frequency
id,27281,27281,cc46051622b8a9c1b883a3bbf12c640b12ac1cbdc7f48a...,1
url,27281,27281,f70a206472e9deaf6e313297c1efb891729ced346a0aeb...,1
source,27281,14,Twitter Web App,14920
createdAt,27281,25085,2024-09-16 21:47:00,26
lang,27281,1,en,27281
isReply,27281,2,False,23911
isRetweet,27281,1,False,27281
isQuote,27281,2,False,24046
isConversationControlled,27281,2,False,27273
month_year,27281,15,2024-10,3582


,source,retweetCount_count,retweetCount_mean,retweetCount_min,retweetCount_max,retweetCount_std_dev,replyCount_count,replyCount_mean,replyCount_min,replyCount_max,...,freefair_illuminating_count,freefair_illuminating_mean,freefair_illuminating_min,freefair_illuminating_max,freefair_illuminating_std_dev,fraud_illuminating_count,fraud_illuminating_mean,fraud_illuminating_min,fraud_illuminating_max,fraud_illuminating_std_dev
0,Twitter for iPhone,8482,613.228130,0.0,67113.0,2157.247708,8482,290.058595,0.0,30215.0,...,8482,0.000943,0.0,1.0,0.030697,8482,0.003419,0.0,1.0,0.058372
1,Twitter Web App,14920,1316.789276,0.0,144615.0,3571.750631,14920,719.317627,0.0,121270.0,...,14920,0.001408,0.0,1.0,0.037490,14920,0.002815,0.0,1.0,0.052982
2,Sprout Social,2933,3524.793386,13.0,80331.0,4583.902650,2933,5186.272076,12.0,60598.0,...,2933,0.002387,0.0,1.0,0.048795,2933,0.000682,0.0,1.0,0.026104
3,Twitter Media Studio,498,1589.315261,8.0,32636.0,3154.603440,498,1164.614458,2.0,18819.0,...,498,0.006024,0.0,1.0,0.077381,498,0.002008,0.0,1.0,0.044766
4,Periscope,103,596.728155,23.0,29648.0,2935.382359,103,402.058252,3.0,14559.0,...,103,0.000000,0.0,0.0,0.000000,103,0.000000,0.0,0.0,0.000000
5,Twitter for Advertisers,7,97.000000,14.0,289.0,89.316452,7,138.714286,31.0,271.0,...,7,0.000000,0.0,0.0,0.000000,7,0.000000,0.0,0.0,0.000000
6,Loomly,10,25.400000,0.0,230.0,68.391812,10,23.100000,0.0,188.0,...,10,0.000000,0.0,0.0,0.000000,10,0.000000,0.0,0.0,0.000000
7,Twitter for Android,5,10.600000,6.0,18.0,5.276362,5,8.400000,4.0,18.0,...,5,0.000000,0.0,0.0,0.000000,5,0.000000,0.0,0.0,0.000000
8,Twitter for iPad,266,65.375940,0.0,723.0,67.843152,266,21.300752,0.0,352.0,...,266,0.000000,0.0,0.0,0.000000,266,0.000000,0.0,0.0,0.000000
9,TweetDeck Web App,7,1911.142857,190.0,7599.0,2433.030763,7,659.285714,42.0,2449.0,...,7,0.000000,0.0,0.0,0.000000,7,0.000000,0.0,0.0,0.000000


,source,id,retweetCount_count,retweetCount_mean,retweetCount_min,retweetCount_max,retweetCount_std_dev,replyCount_count,replyCount_mean,replyCount_min,...,freefair_illuminating_count,freefair_illuminating_mean,freefair_illuminating_min,freefair_illuminating_max,freefair_illuminating_std_dev,fraud_illuminating_count,fraud_illuminating_mean,fraud_illuminating_min,fraud_illuminating_max,fraud_illuminating_std_dev
0,Twitter for iPhone,cc46051622b8a9c1b883a3bbf12c640b12ac1cbdc7f48a...,1,10.0,10.0,10.0,0.0,1,37.0,37.0,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
1,Twitter for iPhone,0e3db0c35a290c6df3b737d15882846c108cc80a9b7e5c...,1,421.0,421.0,421.0,0.0,1,1005.0,1005.0,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
2,Twitter for iPhone,256905919085d2946d5d187abc6cbe81a8abe3384793b3...,1,39.0,39.0,39.0,0.0,1,194.0,194.0,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
3,Twitter for iPhone,a461b32b31e72b222df7fdda0a8e68b0092e31deda33a8...,1,47.0,47.0,47.0,0.0,1,332.0,332.0,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
4,Twitter Web App,ca2795ec79d62adc1fff06c4d3bc9da0bbc899e32c9b21...,1,17.0,17.0,17.0,0.0,1,46.0,46.0,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27276,Twitter Web App,e0f6a1ca295766d8aee817187d9426d3fe23180e50e5e4...,1,180.0,180.0,180.0,0.0,1,74.0,74.0,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
27277,Twitter Web App,df3c096b8981d50713131ffc6324b2e66db7712a2a7b8d...,1,571.0,571.0,571.0,0.0,1,99.0,99.0,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
27278,Twitter Web App,b31abfc148874e3db21fd64857927259795af5d8f622eb...,1,265.0,265.0,265.0,0.0,1,51.0,51.0,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
27279,Twitter Web App,3cc1147c118e42b481ac10ea7dfab7cdcebce938c37bd9...,1,487.0,487.0,487.0,0.0,1,90.0,90.0,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
